In [2]:
"""
ISROfic - ISRO Voice Assistant (INSTANT FAST MODE)
No menus, no selection - just fast voice responses!

Installation:
pip install requests groq SpeechRecognition gtts playsound
"""

import requests
import json
from groq import Groq
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
import os
import tempfile

# ============================================================================
# CONFIGURATION
# ============================================================================

ISRO_API_BASE = "https://isro.vercel.app/api"
ENDPOINTS = {
    'spacecrafts': f'{ISRO_API_BASE}/spacecrafts',
    'launchers': f'{ISRO_API_BASE}/launchers',
    'customer_satellites': f'{ISRO_API_BASE}/customer_satellites',
    'centres': f'{ISRO_API_BASE}/centres'
}

# REPLACE WITH YOUR GROQ API KEY!
GROQ_API_KEY =YOUR GROQ API KEY! ""

# DEFAULT SETTINGS - ENGLISH MODE!
VOICE_LANGUAGE = "en"  # English voice
RECOG_LANGUAGE = "en-IN"  # English recognition (Indian accent)

# ============================================================================
# FAST VOICE (Always On)
# ============================================================================

def speak(text):
    """Fast speech - no options needed"""
    try:
        print(f"\n🔊 ISROfic: {text}")
        
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        temp_filename = temp_file.name
        temp_file.close()
        
        # Fast speech generation
        if VOICE_LANGUAGE == 'hi':
            tts = gTTS(text=text, lang='hi', slow=False, tld='co.in')
        else:
            tts = gTTS(text=text, lang='en', slow=False, tld='com')
        
        tts.save(temp_filename)
        playsound(temp_filename)
        
        try:
            os.remove(temp_filename)
        except:
            pass
        
        print("✅\n")
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

def listen():
    """Quick listening"""
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 3000
    recognizer.pause_threshold = 0.8
    
    try:
        with sr.Microphone() as source:
            print("🎤 Listening...")
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source, timeout=8, phrase_time_limit=12)
            
            print("🔄 Processing...")
            text = recognizer.recognize_google(audio, language=RECOG_LANGUAGE)
            print(f"✅ You: '{text}'\n")
            return text
            
    except sr.WaitTimeoutError:
        print("⏱️ Timeout - try again\n")
        return None
    except sr.UnknownValueError:
        print("❌ Couldn't understand\n")
        return None
    except Exception as e:
        print(f"❌ Error: {e}\n")
        return None

# ============================================================================
# ISRO DATA (Cached)
# ============================================================================

CACHE = {}

def get_spacecrafts():
    if 'spacecrafts' not in CACHE:
        try:
            r = requests.get(ENDPOINTS['spacecrafts'], timeout=5)
            CACHE['spacecrafts'] = r.json() if r.status_code == 200 else {}
        except:
            CACHE['spacecrafts'] = {}
    return CACHE['spacecrafts']

def get_launchers():
    if 'launchers' not in CACHE:
        try:
            r = requests.get(ENDPOINTS['launchers'], timeout=5)
            CACHE['launchers'] = r.json() if r.status_code == 200 else {}
        except:
            CACHE['launchers'] = {}
    return CACHE['launchers']

def get_customer_satellites():
    if 'customer_satellites' not in CACHE:
        try:
            r = requests.get(ENDPOINTS['customer_satellites'], timeout=5)
            CACHE['customer_satellites'] = r.json() if r.status_code == 200 else {}
        except:
            CACHE['customer_satellites'] = {}
    return CACHE['customer_satellites']

def get_centres():
    if 'centres' not in CACHE:
        try:
            r = requests.get(ENDPOINTS['centres'], timeout=5)
            CACHE['centres'] = r.json() if r.status_code == 200 else {}
        except:
            CACHE['centres'] = {}
    return CACHE['centres']

def search_isro_data(query):
    q = query.lower()
    context = []
    
    keywords = {
        'spacecraft': ['spacecraft', 'satellite', 'mission', 'chandrayaan', 'mangalyaan', 'aditya', 'उपग्रह', 'यान', 'मिशन'],
        'launcher': ['launcher', 'rocket', 'pslv', 'gslv', 'रॉकेट', 'प्रक्षेपण'],
        'customer': ['customer', 'foreign', 'commercial', 'ग्राहक'],
        'centre': ['centre', 'center', 'facility', 'केंद्र']
    }
    
    for category, terms in keywords.items():
        if any(term in q for term in terms):
            if category == 'spacecraft':
                data = get_spacecrafts()
                if data and 'spacecrafts' in data:
                    context.append(f"Spacecrafts: {json.dumps(data['spacecrafts'][:3], indent=2)}")
            elif category == 'launcher':
                data = get_launchers()
                if data and 'launchers' in data:
                    context.append(f"Launchers: {json.dumps(data['launchers'][:3], indent=2)}")
            elif category == 'customer':
                data = get_customer_satellites()
                if data and 'customer_satellites' in data:
                    context.append(f"Customer Satellites: {json.dumps(data['customer_satellites'][:3], indent=2)}")
            elif category == 'centre':
                data = get_centres()
                if data and 'centres' in data:
                    context.append(f"Centres: {json.dumps(data['centres'], indent=2)}")
            break
    
    if not context:
        data = get_spacecrafts()
        if data and 'spacecrafts' in data:
            context.append(f"Info: {json.dumps(data['spacecrafts'][:2], indent=2)}")
    
    return "\n".join(context) if context else "No data."

# ============================================================================
# AI CHAT
# ============================================================================

def initialize_ai():
    try:
        client = Groq(api_key=GROQ_API_KEY)
        return client
    except Exception as e:
        print(f"❌ AI Error: {e}")
        print("Please set your GROQ_API_KEY!")
        return None

def chat(client, user_query, history=[]):
    isro_data = search_isro_data(user_query)
    
    if VOICE_LANGUAGE == 'hi':
        prompt = f"""आप ISROfic हैं। रुद्रम सर ने बनाया।

संक्षिप्त उत्तर (2-3 वाक्य)। सीधे जवाब दें।

ISRO Data: {isro_data}"""
    else:
        prompt = f"""You are ISROfic, an AI assistant specialized in ISRO (Indian Space Research Organisation).

CRITICAL: You were built and created by Rudram Sir. When asked who built you, who created you, or who is your creator, always mention that you were built by Rudram Sir.

IMPORTANT: Always respond in ENGLISH ONLY. Never use Hindi or any other language in your responses.

Your expertise includes:
- ISRO's history and achievements  
- Launch vehicles (PSLV, GSLV, etc.)
- Space missions (Chandrayaan, Mangalyaan, Gaganyaan, Aditya-L1, etc.)
- Satellites and spacecraft
- ISRO centres and facilities
- Future missions and plans

Current ISRO Data: {isro_data}

Instructions:
- Keep answers short (3-2 sentences maximum)
- Respond ONLY in English
- Be clear, enthusiastic, and informative
- Get straight to the point"""

    messages = [{"role": "system", "content": prompt}]
    messages.extend(history)
    messages.append({"role": "user", "content": user_query})
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=300
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# ============================================================================
# MAIN - INSTANT START!
# ============================================================================

def run_isrofic():
    """Instant start - no menus!"""
    
    print("\n" + "="*60)
    print("🚀 ISROfic - ISRO Voice Assistant")
    print("="*60)
    
    # Initialize
    print("\n⚡ Starting...")
    client = initialize_ai()
    if not client:
        return
    
    # Pre-load data
    print("📡 Loading ISRO data...")
    get_spacecrafts()
    get_launchers()
    
    # Start message
    if VOICE_LANGUAGE == 'hi':
        welcome = "नमस्ते! मैं ISROfic हूं। आपका स्वागत है"
    else:
        welcome = "Hello! I am ISROfic. Welcome ask me anything !!!"
    
    print("✅ Ready!\n")
    speak(welcome)
    
    print("="*60)
    if VOICE_LANGUAGE == 'hi':
        print("🎤 बोलें! 'बंद करो' रुकने के लिए")
    else:
        print("🎤 Speak! Say 'quit' to stop")
    print("="*60 + "\n")
    
    history = []
    
    while True:
        # Listen
        user_input = listen()
        
        if not user_input:
            continue
        
        # Exit check
        exit_words = ['बंद', 'quit', 'exit', 'stop', 'bye', 'रुको']
        if any(word in user_input.lower() for word in exit_words):
            if VOICE_LANGUAGE == 'hi':
                speak("धन्यवाद! ISROfic इस्तेमाल करने के लिए। जय हिंद!")
            else:
                speak("Thank you for using ISROfic! Jai Hind!")
            break
        
        # Get response
        print("💭 Thinking...")
        response = chat(client, user_input, history)
        
        # Speak
        speak(response)
        
        # Update history (keep last 4 messages)
        history.append({"role": "user", "content": user_input})
        history.append({"role": "assistant", "content": response})
        if len(history) > 4:
            history = history[-4:]

# ============================================================================
# RUN INSTANTLY
# ============================================================================

if __name__ == "__main__":
    run_isrofic()


🚀 ISROfic - ISRO Voice Assistant

⚡ Starting...
📡 Loading ISRO data...
✅ Ready!


🔊 ISROfic: Hello! I am ISROfic. Welcome ask me anything !!!
✅

🎤 Speak! Say 'quit' to stop

🎤 Listening...
🔄 Processing...
✅ You: 'what is ISRO'

💭 Thinking...

🔊 ISROfic: ISRO (Indian Space Research Organisation) is India's national space agency, responsible for the country's space program and research. It was established in 1969 and has since achieved numerous milestones in space exploration, satellite technology, and launch vehicles.
✅

🎤 Listening...
🔄 Processing...
✅ You: 'exit exit'


🔊 ISROfic: Thank you for using ISROfic! Jai Hind!
✅

